In [5]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [6]:
class MF():
    
    def __init__(self, R, K, alpha, beta, iterations):
        """
        Perform matrix factorization to predict empty
        entries in a matrix.
        
        Arguments
        - R (ndarray)   : user-item rating matrix
        - K (int)       : number of latent dimensions
        - alpha (float) : learning rate
        - beta (float)  : regularization parameter
        """
        
        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations

    def train(self):
        # Initialize user and item latent feature matrice
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))
        
        # Initialize the biases
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])
        
        # Create a list of training samples
        self.samples = [
            (i, j, self.R[i, j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
        ]
        
        # Perform stochastic gradient descent for number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            training_process.append((i, mse))
            if (i+1) % 10 == 0:
                print("Iteration: %d ; error = %.4f" % (i+1, mse))
        
        return training_process

    def mse(self):
        """
        A function to compute the total mean square error
        """
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error)

    def sgd(self):
        """
        Perform stochastic graident descent
        """
        for i, j, r in self.samples:
            # Computer prediction and error
            prediction = self.get_rating(i, j)
            e = (r - prediction)
            
            # Update biases
            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])
            
            # Update user and item latent feature matrices
            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])

    def get_rating(self, i, j):
        """
        Get the predicted rating of user i and item j
        """
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction
    
    def full_matrix(self):
        """
        Computer the full matrix using the resultant biases, P and Q
        """
        return mf.b + mf.b_u[:,np.newaxis] + mf.b_i[np.newaxis:,] + mf.P.dot(mf.Q.T)

In [7]:
ratings = [i.strip().split(",") for i in open('data/ratings.csv', 'r').readlines()]
ratings_df = pd.DataFrame(ratings, columns = ['UserID', 'ItemID', 'Rating', 'Timestamp'], dtype = int)
ratings_df[['Rating']] = ratings_df[['Rating']].apply(pd.to_numeric)
R_df = ratings_df.pivot(index = 'UserID', columns ='ItemID', values = 'Rating').fillna(0)
R = R_df.as_matrix()

mf = MF(R, K=2, alpha=0.1, beta=0.01, iterations=20)
training_process = mf.train()
print()
print("P x Q:")
print(mf.full_matrix())
print()
print("Global bias:")
print(mf.b)
print()
print("User bias:")
print(mf.b_u)
print()
print("Item bias:")
print(mf.b_i)

/home/gustavo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:78: RuntimeWarning: overflow encountered in multiply
/home/gustavo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:78: RuntimeWarning: invalid value encountered in subtract
/home/gustavo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:91: RuntimeWarning: invalid value encountered in add


Iteration: 10 ; error = nan
Iteration: 20 ; error = nan

P x Q:
[[ 5.17854231  4.0330065   3.71977341 ...,  4.93560597  4.56053     4.44865273]
 [        nan         nan         nan ...,         nan         nan
          nan]
 [ 4.69547585  3.22420166  4.59230425 ...,  4.65206148  3.9625595
   4.98401853]
 ..., 
 [ 5.31311936  3.8918895   3.88308889 ...,  4.77145253  4.38377315
   4.79437315]
 [ 3.63786794  3.47418256  4.60362845 ...,  5.51511949  4.61224163
   3.62621292]
 [        nan         nan         nan ...,         nan         nan
          nan]]

Global bias:
4.10971428571

User bias:
[ 0.07509623         nan  0.02975796 ...,  0.02854405  0.42556058
         nan]

Item bias:
[ 0.10746907 -0.97072846  0.3491994  -1.01549676  0.29118005 -0.77894788
  0.35712179 -0.60727808 -0.19991397  0.30553757 -0.08449998  0.47042569
 -0.16629363  0.35997516 -0.20772117 -0.48640304  0.35118104  0.14380704
  0.08438475  0.36686451  0.10597828  0.29637422  0.18995393  0.50741797
  0.52576046  0

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  5.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [4]:
mf.get_rating(0, 0)

4.991801088787482